# Project Statement:

## Marina Pier Inc. is leveraging technology to automate their operations on the San Francisco port.

## The company’s management has set out to build a bias-free/ corruption-free automatic system that reports & avoids faulty situations caused by human error. Examples of human error include misclassifying the correct type of boat. The type of boat that enters the port region is as follows

* Buoy
* Cruise_ship
* Ferry_boat
* Freight_boar
* Gondola
* Inflatable_boat
* Kayak
* Paper_boat
* Sailboat

## Marina Pier wants to use Deep Learning techniques to build an automatic reporting system that recognizes the boat. The company is also looking to use a transfer learning approach of any lightweight pre-trained model in order to deploy in mobile devices.

## As a deep learning engineer, our task is to:

1. Build a CNN network to classify the boat.
2. Build a lightweight model with the aim of deploying the solution on a mobile device using transfer learning. We can use any lightweight pre-trained model as the initial (first) layer. MobileNetV2 is a popular lightweight pre-trained model built using Keras API.

# Dataset and Data Description: 

## The dataset folder is called Automating_Port_Operations_dataset 

## The dataset contains images of 9 types of boats. It contains a total of 1162 images. The training images are provided in the directory of the specific class itself. 
## Classes:
* ferry_boat
* gondola
* sailboat
* cruise_ship
* kayak
* inflatable_boat
* paper_boat
* buoy
* freight_boat




# 1. Building a CNN to classify the boat

In [ ]:
#pip install tensorflow

# Importing necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D


from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix

import os
import PIL
from PIL import Image
import pathlib
import glob

# Defining the path for the dataset and displaying first image from each image directory

In [ ]:

# Defining base path where dataset is located
data_dir = os.path.join(os.getcwd(), 'Automating_Port_Operations_dataset')

# Getting only visible directories (exclude hidden directories like .DS_Store)
image_dirs = [d for d in os.listdir(data_dir) if not d.startswith('.')]

print(image_dirs) # List of directories in the dataset
print(data_dir) # Path to parent directory of dataset
print()

# Function to display single image given an image path
def display_image(image_path):
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    
# Displaying the first image in each directory in image_dirs
for d in image_dirs:
    image_path = os.path.join(data_dir, d, os.listdir(os.path.join(data_dir, d))[0])
    print(f"Displaying first image from folder: {d}")
    display_image(image_path)


# Creating training and datasets with following characteristics:

* Dataset will be split into train and test in the ratio 80:20, with shuffle and random state = 43
* We'll use tf.keras.preprocessing.image_dataset_from_directory to load the train and test datasets with data normalization

* We'll load train, validation and test dataset in batches of 32 using the function initialized in step above

## Dataset Split into Training, Validation and Testing with normalzation

## Visualizing images from training dataset

# Create Convolutional Neural Network Model

# We'll build a CNN network with Keras with the following layers:

### We'll do data augmentation issue to balance the data as we have some images that are 

### Conv2D with 32 filters, kernel size 3,3, and activation relu, followed by MaxPool2D
### Conv2D with 32 filters, kernel size 3,3, and activation relu, followed by MaxPool2D
### GLobalAveragePooling2D layer
### Dense layer with 128 neurons and activation relu
### Dense layer with 128 neurons and activation relu
### Dense layer with 9 neurons and activation softmax.


Compile the model with Adam optimizer, categorical_crossentropy loss, and with metrics accuracy, precision, and recall.
Train the model for 20 epochs and plot training loss and accuracy against epochs.
Evaluate the model on test images and print the test loss and accuracy.
Plot heatmap of the confusion matrix and print classification report.


In [ ]:


# Set parameters
batch_size = 32
img_height = 150
img_width = 150
AUTOTUNE = tf.data.AUTOTUNE

# Define data directory
data_dir = os.path.join(os.getcwd(), 'Automating_Port_Operations_dataset')

# Create training dataset
train_val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,  # 80% for training, 20% for validation + test
    subset="training",
    seed=43,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True
)

# Create validation and test datasets
val_test_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2, # 20% for validation + test 
    subset="validation",
    seed=43,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_val_ds.class_names  # Get class names    
print(f"Class names: {class_names}")

# Further split val_test_ds into validation and test sets
val_batches = int(0.5 * len(val_test_ds)) # Split the validation and test sets in half

# Isolate the validation and test datasets
val_ds = val_test_ds.take(val_batches)
test_ds = val_test_ds.skip(val_batches)

# Normalize the pixel values
def normalize_img(image, label):
    image = image / 255.0
    return image, label

def one_hot_encode(image, label):
    label = tf.one_hot(label, depth=len(class_names))  # One-hot encode the labels (convert to binary vectors) in order to use categorical crossentropy loss
    return image, label

# Apply this to the datasets
train_ds = train_val_ds.map(lambda x, y: (x / 255.0, tf.one_hot(y, len(class_names)))).cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE) # Cache, shuffle, and prefetch the training dataset
val_ds = val_ds.map(lambda x, y: (x / 255.0, tf.one_hot(y, len(class_names)))).cache().prefetch(buffer_size=AUTOTUNE) # Cache and prefetch the validation dataset
test_ds = test_ds.map(lambda x, y: (x / 255.0, tf.one_hot(y, len(class_names)))).cache().prefetch(buffer_size=AUTOTUNE) # Cache and prefetch the test dataset






## Create model

In [ ]:


input_shape = (img_height, img_width, 3) # Define the input shape because we are using a Sequential model

model = models.Sequential([
    layers.Input(shape=input_shape),  # Explicit Input layer to define the input shape
    layers.Conv2D(32, (3, 3), activation='relu'), # Convolutional layer with 32 filters, 3x3 kernel size, and ReLU activation
    layers.MaxPooling2D(), # Max pooling layer
    layers.Conv2D(32, (3, 3), activation='relu'), # Convolutional layer with 32 filters, 3x3 kernel size, and ReLU activation
    layers.MaxPooling2D(), # Max pooling layer
    layers.GlobalAveragePooling2D(), # Global average pooling layer which averages the values in the feature maps
    layers.Flatten(), # Flatten layer to flatten the output of the previous layer
    layers.Dense(128, activation='relu'), # Dense layer with 128 units and ReLU activation
    layers.Dense(128, activation='relu'), # Dense layer with 128 units and ReLU activation
    layers.Dense(len(class_names), activation='softmax') # Output layer with units equal to the number of classes and softmax activation
])

model.summary() # Display the model summary



## Compile the model

In [ ]:
model.compile(
    optimizer='adam', # Adam optimizer
    loss='categorical_crossentropy', # Categorical crossentropy loss
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()] # Accuracy, precision, and recall metrics
)



## Train the model

In [ ]:
# Train the model
# Set the number of epochs to 20
epochs = 20

# Train the model
history = model.fit(
    train_ds, # Use the training dataset for training
    validation_data=val_ds,  # Use the validation dataset for validation during training
    epochs=epochs # Set the number of epochs
)



## Plot Training and Validation Accuracy/Loss over epochs

In [ ]:
# Plot the training and validation accuracy/loss over epochs
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs) # Define the range of epochs

plt.figure(figsize=(12, 6)) # Set the figure size

# Plot training and validation accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()


## Model Evaluation

In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_ds)

# Print test loss, accuracy, precision, and recall
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
print(f'Test Precision: {test_precision}')
print(f'Test Recall: {test_recall}')




## Evaluate model on test images and print test loss and accuracy

In [ ]:
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Generate predictions from the model on the test set
y_pred = []
y_true = []

# Iterate over the test dataset to get true labels and predicted labels
for images, labels in test_ds:
    predictions = model.predict(images)
    y_pred.extend(np.argmax(predictions, axis=1))  # Convert predicted probabilities to class labels
    y_true.extend(np.argmax(labels.numpy(), axis=1))  # Convert one-hot encoded labels back to integers



## Ploat heatmap of confusion matrix and print classification report

In [ ]:
# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Print the classification report
print('Classification Report:')
print(classification_report(y_true, y_pred, target_names=class_names))


# 2. Build a lightweight model with the aim of deploying the solution on a mobile device using transfer learning, by using MobileNetV2, a lightweight pre-trained model using Keras API as the initial (first) layer.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
import os
import matplotlib.pyplot as plt

# Set parameters
batch_size = 32
img_height = 224  # Updated to match MobileNetV2 expected input size
img_width = 224
AUTOTUNE = tf.data.AUTOTUNE

# Define data directory
data_dir = os.path.join(os.getcwd(), 'Automating_Port_Operations_dataset')

# Function to normalize the pixel values and one-hot encode labels
def one_hot_encode(image, label):
    label = tf.one_hot(label, depth=9)  # Assuming 9 classes in the dataset
    image = image / 255.0  # Normalize the image to the range [0, 1]
    return image, label

# Load the dataset and split it into train and validation (70% train, 30% validation)
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="training",
    seed=1,
    image_size=(img_height, img_width),  # Set to 224x224 to match MobileNetV2
    batch_size=batch_size,
    shuffle=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="validation",
    seed=1,
    image_size=(img_height, img_width),  # Set to 224x224 to match MobileNetV2
    batch_size=batch_size
)

# Apply normalization and one-hot encoding to the datasets
train_ds = train_ds.map(one_hot_encode).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(one_hot_encode).cache().prefetch(buffer_size=AUTOTUNE)

# Load the MobileNetV2 pre-trained model with weights from ImageNet
base_model = MobileNetV2(input_shape=(img_height, img_width, 3),
                         include_top=False,  # We don't need the top layers (classification layers)
                         weights='imagenet')  # Load pre-trained ImageNet weights

# Freeze the base model
base_model.trainable = False

# Build the CNN model
model = models.Sequential([
    base_model,  # Add MobileNetV2 as the base layer
    layers.GlobalAveragePooling2D(),  # Reduce dimensionality
    layers.Dropout(0.2),  # Dropout for regularization
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),
    layers.Dense(9, activation='softmax')  # Assuming 9 output classes (one-hot encoded)
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Use categorical crossentropy for one-hot encoded labels
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Display the model summary
#model.summary()

# Early stopping callback to avoid overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[early_stopping]
)

# Evaluate the model on the validation set
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(val_ds)

# Print evaluation results
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")

# Plot training and validation accuracy and loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 6))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Training vs Validation Accuracy')

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Training vs Validation Loss')

plt.show()


## Split Dataset (Train/Test 70:30) using tf.keras.preporcessing.image_dataset_from_directory to load and split the dataset, shuffle the data, and set a random seed to ensure consistency.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
import os
import matplotlib.pyplot as plt

# Set parameters
batch_size = 32
img_height = 224  # Updated to match MobileNetV2 expected input size
img_width = 224
AUTOTUNE = tf.data.AUTOTUNE

# Define data directory
data_dir = os.path.join(os.getcwd(), 'Automating_Port_Operations_dataset')


## Normalize dataset pixel values and one-hot encode labels

In [ ]:
# Function to normalize the pixel values and one-hot encode labels
def one_hot_encode(image, label):
    label = tf.one_hot(label, depth=9)  # Assuming 9 classes in the dataset
    image = image / 255.0  # Normalize the image to the range [0, 1]
    return image, label

# Load the dataset and split it into train and validation (70% train, 30% validation)
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="training",
    seed=1,
    image_size=(img_height, img_width),  # Set to 224x224 to match MobileNetV2
    batch_size=batch_size,
    shuffle=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="validation",
    seed=1,
    image_size=(img_height, img_width),  # Set to 224x224 to match MobileNetV2
    batch_size=batch_size
)

# Apply normalization and one-hot encoding to the datasets
train_ds = train_ds.map(one_hot_encode).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(one_hot_encode).cache().prefetch(buffer_size=AUTOTUNE)

## Building the CNN with Transfer Learning Using MobileNetV2

In [ ]:
# Load the MobileNetV2 pre-trained model with weights from ImageNet
base_model = MobileNetV2(input_shape=(img_height, img_width, 3),
                         include_top=False,  # We don't need the top layers (classification layers)
                         weights='imagenet')  # Load pre-trained ImageNet weights

# Freeze the base model
base_model.trainable = False

# Build the CNN model
model = models.Sequential([
    base_model,  # Add MobileNetV2 as the base layer
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),  # Dropout for regularization
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),
    layers.Dense(9, activation='softmax')  # Assuming 9 output classes
])



## Compile the model

In [ ]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Use categorical crossentropy for one-hot encoded labels
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Display the model summary
####model.summary()

In [ ]:
# Early stopping callback to avoid overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[early_stopping]
)


In [ ]:
# Evaluate the model on the validation set
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(val_ds)

# Print evaluation results
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")


In [ ]:
# Plot training and validation accuracy and loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 6))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Training vs Validation Accuracy')

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Training vs Validation Loss')

plt.show()

In [ ]:
# Compute confusion matrix and plot it as a heatmap
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Generate predictions from the model on the validation set
y_pred = []
y_true = []

# Iterate over the validation dataset to get true labels and predicted labels
for images, labels in val_ds:
    predictions = model.predict(images)
    y_pred.extend(np.argmax(predictions, axis=1))  # Convert predicted probabilities to class labels
    y_true.extend(np.argmax(labels.numpy(), axis=1))  # Convert one-hot encoded labels back to integers 
    
# Compute the confusion matrix  
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Print the classification report
print('Classification Report:')
print(classification_report(y_true, y_pred, target_names=class_names))


